In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import string
from sklearn.feature_extraction import stop_words
import operator

In [2]:
data = pd.read_csv('GSAF5.csv', encoding = 'latin')
data.head(5)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [3]:
# suppression des espaces, y compris en milieu de chaine
def supprespace(x):
    if type(x) is str:
        return re.sub(' +', ' ',(x.strip()))
    return x
data = data.applymap(supprespace)

# suppression des éventuels doublons
data = data.drop_duplicates() #(subset = ['colonne(s) sur laquelle baser le drop'])

In [4]:
#évalution taille fichier
print(data['Case Number'].count())

5992


In [5]:
#vérifie qu'il n'y a pas de colonnes vides
null_cols = data.isnull().sum()
pourcent_col = null_cols / data['Case Number'].count()*100
print("Poids des valeurs nulles dans chacun des champs")
print(pourcent_col)

Poids des valeurs nulles dans chacun des champs
Case Number                0.000000
Date                       0.000000
Year                       0.000000
Type                       0.000000
Country                    0.717623
Area                       6.708945
Location                   8.277704
Activity                   8.795060
Name                       3.337784
Sex                        9.462617
Age                       44.742991
Injury                     0.450601
Fatal (Y/N)                0.317089
Time                      53.621495
Species                   48.965287
Investigator or Source     0.250334
pdf                        0.000000
href formula               0.016689
href                       0.050067
Case Number.1              0.000000
Case Number.2              0.000000
original order             0.000000
Unnamed: 22               99.983311
Unnamed: 23               99.966622
dtype: float64


In [6]:
## COLONNES PRESQUE VIDES
#checke le contenu des colonnes presque vides, pour vérifier qu'on peut les supprimer
data[data['Unnamed: 22'].fillna(0) != 0]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
1247,2006.06.18,18-Jun-06,2006,Unprovoked,BRAZIL,Pernambuco,"Punta Del Chifre Beach, Olinda",Body boarding,Humberto Pessoa Batista,M,...,NaN,globalsurfnews.com,2006.06.18-Batista.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2006.06.18,2006.06.18,4746,stopped here,NaN


In [7]:
data[data['Unnamed: 23'].fillna(0) != 0]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
4182,1952.07.13,13-Jul-52,1952,Provoked,USA,California,"San Diego, San Diego County",Fishing,"Gerald Howard, on board sportsfishing boat Ter...",M,...,NaN,"L.A. Times, 7/14/1952",1952.07.13-Howard.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.07.13,1952.07.13,1811,NaN,Teramo
5585,1879.03.10,10-Mar-1879,1879,Invalid,AUSTRALIA,New South Wales,Near Sydney,The steamship Bonnie Dundee lost in collision,Cabin boy of the Bonnie Dundee,M,...,NaN,"Star, 3/22/1879",1879.03.10-Bonnie-Dundee.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1879.03.10,1879.03.10,408,NaN,change filename


In [8]:
#Etant donné le résultat, on supprimera les colonnes.

In [9]:
## COLONNES HREF, HREF FORMULA, PDF
#les colonnes href, href formula et pdf contiennent des infos redondantes.
#Comparaison des colonnes pour sélectionner laquelle conserver.

#reconstitue un URL à partir de la colonne pdf
urlreconstitue = 'http://sharkattackfile.net/spreadsheets/pdf_directory/' + data['pdf']
#compare cet URL reconstitué avec les contenus des colonnes href et href formula
filthref = ~data['href'].eq(urlreconstitue)
filtformula = ~data['href formula'].eq(urlreconstitue)

#calcul de quelques indicateurs de propreté des champs href
FauxvsPDFhref = data[filthref].size
OKformathref = data['href'].astype(str).apply(lambda x: not x.endswith('.pdf'))
valueshref = data[OKformathref].size

#calcul de quelques indicateurs de propreté du champ href formula
FauxvsPDFformula = data[filtformula].size
OKformatformula = data['href formula'].astype(str).apply(lambda x: not x.endswith('.pdf'))
valuesformula = data[OKformatformula].size

#comparaison de la propreté des 2 champs
print("nb val incohérentes entre le champ pdf et le champ href : " + str(FauxvsPDFhref) + " vs " + str(FauxvsPDFformula) + " avec le champ href formula")
print("nb val qui terminent correctement : " + str(valueshref) + " dans le champ href, vs " + str(valuesformula) + " dans le champ href formula")

nb val incohérentes entre le champ pdf et le champ href : 1296 vs 120 avec le champ href formula
nb val qui terminent correctement : 840 dans le champ href, vs 672 dans le champ href formula


In [10]:
## Compte tenu des résultats précédents : on garde le champ href formula.
## => nettoyage de cette colonne avant suppression des autres.

# compléter les cellules vides de href formula par la valeur de href
data['href'] = data['href formula'].fillna(data['href'])

# remplacer les valeurs de href formula lorsqu'elles ne commencent pas comme il faut par celles de href
dataf = data['href'].apply(lambda x: not x.startswith('http'))
values = data[dataf]
data.loc[values.index, 'href formula'] = data.loc[values.index, 'href']

# remplacer les valeurs de href formula lorsqu'elles ne terminent pas comme il faut par celles de href
dataf = data['href'].apply(lambda x: not x.endswith('.pdf'))
values = data[dataf]
data.loc[values.index, 'href formula'] = data.loc[values.index, 'href']

#estimation du nombre d'améliorations possibles encore présentes
data[filtformula == True].shape[0]
print(data[filtformula == True]['href formula'])

241     http://sharkattackfile.net/spreadsheets/pdf_di...
448                    pdf-directory/2013.05.27.b-Ena.pdf
3019                                                  NaN
5906    Q93http://sharkattackfile.net/spreadsheets/pdf...
5991    http://sharkattackfile.net/spreadsheets/pdf_di...
Name: href formula, dtype: object


In [11]:
# Compte tenu du faible nombre de corrections restantes, qui ne présentent pas de règle commune, on peut terminer à la main

In [12]:
## CHAMP DATE
#récupère les dates en format 01-Janvier-(20)10
data['Datecorr'] = data['Date'].str.extract(r'([0-9]{2}-[A-Za-z]{4,}-[0-9]{2,4})')
#récupère les dates en format 01-Jan-(20)10
data['Datetemp'] = data['Date'].str.extract(r'([0-9]{2}-[A-Za-z]{3}-[0-9]{2,4})')
data['Datecorr'] = data['Datecorr'].fillna(data['Datetemp'])
#récupère les dates en format Jan(vier)-(20)10
data['Datetemp'] = data['Date'].str.extract(r'([A-Za-z]{3,}-[0-9]{2,4})')
data['Datecorr'] = data['Datecorr'].fillna('01-' + data['Datetemp'])

In [13]:
#complète l'année lorsqu'elle est sur 2 positions :
#
def unifdate(date):
    dateunif = []
    for i in date:
        if len(str(i)) == 9:
            i = i.replace('-0','-200').replace('-1','-201')
        if len(str(i)) == 9:
            i = i.replace('-2','-192').replace('-3','-193').replace('-4','-194').replace('-5','-195').replace('-6','-196').replace('-7','-197').replace('-8','-198').replace('-9','-199')
        dateunif.append (i)
    dateformatee = pd.DataFrame(dateunif)
    return dateformatee
data['Datecorr'] = unifdate(data['Datecorr'])
data['Datecorr'] = pd.to_datetime(data['Datecorr'])

In [14]:
#complète les valeurs manquantes par une date fictive
data['Datecorr'] = data['Datecorr'].fillna("01-Jan-2099")

#Conversion du champ en format date.
data['Datecorr'] = pd.to_datetime(data['Datecorr']).dt.strftime('%Y.%m.%d')

In [15]:
Counter(data['Datecorr']).most_common()

[('2099.01.01', 523),
 ('1960.10.01', 6),
 ('1956.08.01', 6),
 ('2003.10.05', 5),
 ('2001.04.12', 5),
 ('1995.07.28', 5),
 ('1965.06.01', 5),
 ('2015.09.20', 4),
 ('2014.12.28', 4),
 ('2012.06.14', 4),
 ('2008.06.26', 4),
 ('2010.01.09', 4),
 ('2009.07.24', 4),
 ('2008.12.27', 4),
 ('2003.07.01', 4),
 ('1994.07.09', 4),
 ('1970.01.23', 4),
 ('1960.04.01', 4),
 ('1952.07.27', 4),
 ('1906.10.10', 4),
 ('2016.09.18', 3),
 ('2016.07.08', 3),
 ('2016.07.07', 3),
 ('2015.10.17', 3),
 ('2015.06.26', 3),
 ('2015.06.24', 3),
 ('2014.08.27', 3),
 ('2014.06.09', 3),
 ('2014.06.01', 3),
 ('2014.05.14', 3),
 ('2014.03.18', 3),
 ('2013.09.21', 3),
 ('2013.09.01', 3),
 ('2013.07.28', 3),
 ('2013.06.25', 3),
 ('2012.09.02', 3),
 ('2012.07.07', 3),
 ('2012.06.26', 3),
 ('2011.08.17', 3),
 ('2011.06.06', 3),
 ('2010.05.18', 3),
 ('2010.02.06', 3),
 ('2009.03.06', 3),
 ('2009.01.24', 3),
 ('2008.08.30', 3),
 ('2008.06.28', 3),
 ('2008.03.28', 3),
 ('2007.09.30', 3),
 ('2007.08.19', 3),
 ('2006.08.20', 3)

In [16]:
## CHAMPS Case Number, Case Number.1 et Case Number.2

# Application d'une première correction pour éliminer les soucis de formatage simples
data['Case Number'] = data['Case Number'].apply(lambda x:x.replace("-",".").replace(",",".").replace("/","."))
data['Case Number.1'] = data['Case Number.1'].apply(lambda x:x.replace("-",".").replace(",",".").replace("/","."))
data['Case Number.2'] = data['Case Number.2'].apply(lambda x:x.replace("-",".").replace(",",".").replace("/","."))
data = data.drop_duplicates()

In [17]:
#dataframe qui ne contient que les différences
corr1 = data[data['Case Number'].eq(data['Case Number.1']) == False]
corr2 = data[data['Case Number'].eq(data['Case Number.2']) == False]
listeerr = [corr1, corr2]
corrCaseNum = pd.concat(listeerr)
corrCaseNum = corrCaseNum.drop_duplicates()
corrCaseNum[['Case Number', 'Case Number.1','Case Number.2', 'Datecorr', 'Date']]

,Case Number,Case Number.1,Case Number.2,Datecorr,Date
4,2016.09.15,2016.09.16,2016.09.15,2016.09.16,16-Sep-16
33,2016.07.14.4,2016.07.14.R,2016.07.14.4,2016.07.14,Reported 14-Jul-2016
97,2016.01.24.b,2015.01.24.b,2016.01.24.b,2016.01.24,24-Jan-16
116,2015.12.23,2015.11.07,2015.12.23,2015.11.07,07-Nov-15
121,2015.10.28.a,2015.10.28,2015.10.28.a,2015.10.28,28-Oct-15
3654,1961.09.02.R,1961.09.06.R,1961.09.02.R,1961.09.06,Reported 06-Sep-1961
4177,1952.08.05,1952.08.04,1952.08.05,1952.08.05,05-Aug-52
4719,1934.01.08.R,1934.02.08.R,1934.02.08.R,1934.02.08,Reported 08-Feb-1934
5043,1900.00.00.R,1919.00.00.R,1900.00.00.R,2099.01.01,Reported to have taken place in 1919
5150,1911.07.31.R,1911.07.31.T,1911.07.31.R,1911.07.31,Reported 31-Jul-1911


In [18]:
#à partir des différences, comparaison des dates dans les différents champs :
#on isole les dates des éventuelles lettres suffixes
corrCaseNum['DateCaseNb'] = corrCaseNum['Case Number'].apply(lambda x: x[:10])
corrCaseNum['DateCaseNb1'] = corrCaseNum['Case Number.1'].apply(lambda x: x[:10])
corrCaseNum['DateCaseNb2'] = corrCaseNum['Case Number.2'].apply(lambda x: x[:10])
#on crée un nouveau dataframe
#corrCaseNum = pd.concat([DateCaseNb, DateCaseNb1, DateCaseNb2, corrCaseNum['datecorr2']], axis = 1)

In [19]:
comptdate = corrCaseNum[['DateCaseNb', 'DateCaseNb1', 'DateCaseNb2', 'Datecorr']].apply(lambda x: Counter(x).most_common(1)[0], axis = 1)
datemaj = [x[0] for x in comptdate]
print(datemaj)
comptdate

['2016.09.15', '2016.07.14', '2016.01.24', '2015.12.23', '2015.10.28', '1961.09.02', '1952.08.05', '1934.02.08', '1900.00.00', '1911.07.31']


4       (2016.09.15, 2)
33      (2016.07.14, 4)
97      (2016.01.24, 3)
116     (2015.12.23, 2)
121     (2015.10.28, 4)
3654    (1961.09.02, 2)
4177    (1952.08.05, 3)
4719    (1934.02.08, 3)
5043    (1900.00.00, 2)
5150    (1911.07.31, 4)
dtype: object

In [20]:
#on vérifie les éventuels doublons sur les dates à étudier.
#Pour ça, on commence par supprimer les dates des ID qu'on veut traiter, pour ne pas les identifier comme des doublons.
vals_case_num = pd.Series(data['Case Number'])
vals_case_num = vals_case_num.drop(comptdate.index)
occurrences = []
for i in comptdate.index:    
    for j in vals_case_num:
        if j[:10] == str(comptdate[i][0][:10]):
            occurrences.append(j)
print(occurrences)

['2016.09.15.R', '2016.01.24.a', '1900.00.00.b', '1900.00.00.a']


In [21]:
#liste qui ne conserve que les valeurs uniques : permet d'identifier les éléments qui nécessitent une vérification.
datesoccur = list(set([x[:10] for x in occurrences]))

#vérifie si la date est identique dans 3 ou 4 des champs et fixe la valeur majoritaire si c'est le cas
# + s'ajuste aux cas particuliers (doublons et date "reported")

for x in comptdate.index:
    if comptdate[x][1] >=3:
# cas des dates "Reported"
        if "Reported" in data.loc[x,'Date']:
            corrCaseNum['Case Number'][x] = str(comptdate[x][0]) + '.R'
            corrCaseNum['Case Number.1'][x] = str(comptdate[x][0]) + '.R'
            corrCaseNum['Case Number.2'][x] = str(comptdate[x][0]) + '.R'

# cas où la date n'est pas déjà présente dans le champ
        elif comptdate[x][0] not in datesoccur :
            corrCaseNum['Case Number'][x] = str(comptdate[x][0])
            corrCaseNum['Case Number.1'][x] = str(comptdate[x][0])
            corrCaseNum['Case Number.2'][x] = str(comptdate[x][0])

# cas où la date est présente : on recrée un suffixe avec une lettre en vérifiant s'il existe ou non.
        else:
            cpt = 97
            while str(comptdate[x][0]) + '.' + chr(cpt) in occurrences:
                cpt +=1
            corrCaseNum['Case Number'][x] = str(comptdate[x][0]) + '.' + chr(cpt)
            corrCaseNum['Case Number.1'][x] = str(comptdate[x][0]) + '.' + chr(cpt)
            corrCaseNum['Case Number.2'][x] = str(comptdate[x][0]) + '.' + chr(cpt)

### Lorsqu'il n'y a pas de date majoritaire, on part du principe que la date du champ Date est la plus fiable
### (sauf lorsqu'elle correspond à une valeur par défaut) :
    elif corrCaseNum['Datecorr'][x] != '2099.01.01':
# cas des dates "Reported"
        if "Reported" in data.loc[x,'Date']:
            corrCaseNum['Case Number'][x] = str(corrCaseNum['Datecorr'][x]) + '.R'
            corrCaseNum['Case Number.1'][x] = str(corrCaseNum['Datecorr'][x]) + '.R'
            corrCaseNum['Case Number.2'][x] = str(corrCaseNum['Datecorr'][x]) + '.R'

# cas où la date n'est pas déjà présente dans le champ
        elif comptdate[x][0] not in datesoccur :
            corrCaseNum['Case Number'][x] = str(corrCaseNum['Datecorr'][x])
            corrCaseNum['Case Number.1'][x] = str(corrCaseNum['Datecorr'][x])
            corrCaseNum['Case Number.2'][x] = str(corrCaseNum['Datecorr'][x])

# cas où la date est présente : on recrée un suffixe avec une lettre en vérifiant s'il existe ou non.
        else:
            cpt = 97
            while str(comptdate[x][0]) + '.' + chr(cpt) in occurrences:
                cpt +=1
            corrCaseNum['Case Number'][x] = str(corrCaseNum['Datecorr'][x]) + '.' + chr(cpt)
            corrCaseNum['Case Number.1'][x] = str(corrCaseNum['Datecorr'][x]) + '.' + chr(cpt)
            corrCaseNum['Case Number.2'][x] = str(corrCaseNum['Datecorr'][x]) + '.' + chr(cpt)

C:\Users\lleki\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\lleki\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\lleki\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\lleki\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [22]:
#dataframe qui ne contient que les différences
corr1 = corrCaseNum[corrCaseNum['Case Number'].eq(corrCaseNum['Case Number.1']) == False]
corr2 = corrCaseNum[corrCaseNum['Case Number'].eq(corrCaseNum['Case Number.2']) == False]
listeerr = [corr1, corr2]
corrCaseNum = pd.concat(listeerr)
corrCaseNum = corrCaseNum.drop_duplicates()
corrCaseNum[['Case Number', 'Case Number.1','Case Number.2', 'Datecorr', 'Date']]

,Case Number,Case Number.1,Case Number.2,Datecorr,Date
5043,1900.00.00.R,1919.00.00.R,1900.00.00.R,2099.01.01,Reported to have taken place in 1919


In [23]:
## on gère cette dernière exception à la main.

In [24]:
## SUPPRESSION DE TOUTES LES COLONNES INUTILES IDENTIFIEES JUSQUE LA
data.drop(['Unnamed: 22', 'Unnamed: 23', 'href formula', 'Date', 'Datetemp','Case Number.1','Case Number.2'], axis = 1, inplace = True)
# renommage de la nouvelle colonne Date
data.rename(columns=lambda x: x.replace('Datecorr', 'Date'), inplace=True)

In [25]:
data.head(5)

,Case Number,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href,original order,Date
0,2016.09.18.c,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5993,2016.09.18
1,2016.09.18.b,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5992,2016.09.18
2,2016.09.18.a,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5991,2016.09.18
3,2016.09.17,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5990,2016.09.17
4,2016.09.15,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5989,2016.09.16


In [27]:
### RESTE A : virer le texte restant + mettre les champs vides ou nuls à 0
#transformer le champ en float => ligne prête à la fin
#virer la première ligne de code

data = pd.read_csv('GSAF5.csv', encoding = 'latin')


## COLONNE AGE

# nettoyage de quelques valeurs récurrentes
data['Age'] = data['Age'].astype(str).apply(lambda x:x.lower().replace('teen','13').replace('0s','5').replace("0's",'5').replace('ca. ','').replace('½','.5').replace(' & ',',').replace(' and ',',').replace('?',''))
data['Age'] = data['Age'].replace(' ','0').replace('m','0').replace('f','0').replace('nan','0').replace('','0').replace('13s','13')
data['Age'] = data['Age'].fillna('0')
data['Age']=[supprespace(str(x)) for x in data['Age']]

#Fonction qui convertit les mois en années
def moisannee (x):
    chiffres = re.findall('\d+',x)
    premchiffre = chiffres[0]
    return (str(float(premchiffre)/12))

#sélection des valeurs en mois, et conversion.
libmonths = data[data['Age'].str.contains("month")]
libmonths['Age'] = libmonths['Age'].apply(moisannee)
data.loc[libmonths.index, 'Age'] = libmonths.loc[libmonths.index, 'Age']

#Fonction qui fait la moyenne des valeurs annoncées
#(traitement des intervalles : "x to y" ou "x or y")
def intervalle (x):
    chiffres = re.findall('\d+',x)
    chiffres = [int(x) for x in chiffres]
    return (str(sum(chiffres)/len(chiffres)))

#Traitement des intervalles
valsintervalle = data[data['Age'].str.contains(" to ")|data['Age'].str.contains(" or ")]
valsintervalle = valsintervalle['Age'].apply(intervalle)
data.loc[valsintervalle.index, 'Age'] = valsintervalle.loc[valsintervalle.index]

def extractchiffres (x):
    chiffres = re.findall('\d+.*\d*',x)
    return (chiffres)
data['Agecorr'] = data['Age'].apply(extractchiffres)
data['Agecorr'] = [x for x in data['Agecorr'].apply(supprespace)]
#data['Age'] = data['Age'].astype(float)
#data['Agetest'] = [float(y) for x in data['Agetest'] for y in x]
for x in data['Agecorr']:
    print(x)

C:\Users\lleki\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['16']
['36']
['43']
['0']
['0']
['0']
['65']
['51']
['50']
['12']
['0']
['9']
['22']
['25']
['37']
['20']
['49']
['15']
['21']
['22']
['40']
['72']
['0']
['43']
['18']
['29']
['36']
['0']
['31']
['0']
['9']
['11']
['0']
['0']
['10']
['0']
['59']
['0']
['42']
['34']
['35']
['11']
['59']
['43']
['42']
['0']
['0']
['19']
['6']
['19']
['27']
['64']
['60']
['23']
['0']
['0']
['29']
['52']
['13']
['57']
['22']
['11']
['48']
['23']
['59']
['39']
['24']
['15']
['0']
['12']
['26']
['26']
['69']
['60']
['34']
['0']
['46']
['22']
['13']
['41']
['9']
['0']
['19']
['45']
['12']
['0']
['41']
['21']
['31']
['0']
['65']
['0']
['45']
['40']
['0']
['0']
['15']
['0']
['0']
['48']
['0']
['11']
['31']
['0']
['38']
['71']
['43']
['32']
['58']
['0']
['26']
['22']
['0']
['0']
['28']
['20']
['9']
['54']
['28']
['19']
['50']
['10']
['42']
['21']
['21']
['35']
['32']
['44']
['0']
['9']
['25']
['0']
['18']
['22']
['13']
['14']
['22']
['7']
['0']
['14']
['27']
['12']
['16']
['0']
['20']
['42']
['50']
['29']
['62'

['19']
['22']
['0']
['16']
['46']
['0']
['26']
['12']
['40']
['0']
['0']
['12']
['20']
['0']
['0']
['0']
['40']
['35']
['55']
['0']
['0']
['0']
['23']
['47']
['0']
['30']
['38']
['0']
['28']
['47']
['73']
['27']
['52']
['49']
['27']
['31']
['23']
['56']
['69']
['17']
['23']
['11']
['24']
['0']
['17']
['25']
['12']
['31']
['0']
['49']
['26']
['25']
['8']
['48']
['28']
['40']
['27']
['0']
['47']
['69']
['0']
['0']
['53']
['27']
['44']
['0']
['23,20']
['0']
['5']
['45']
['16']
['14']
['15']
['18']
['0']
['69']
['13']
['41']
['5']
['12']
['20']
['55']
['13']
['19']
['0']
['0']
['0']
['52']
['0']
['17']
['55']
['44']
['13']
['0']
['0']
['0']
['9']
['0']
['34']
['0']
['0']
['26']
['14']
['37']
['35']
['0']
['0']
['0']
['27']
['0']
['39']
['27']
['0']
['0']
['12']
['26']
['0']
['35']
['0']
['15']
['13']
['12']
['18']
['0']
['39']
['35']
['19']
['51']
['0']
['35']
['39']
['12']
['12']
['45']
['13']
['0']
['16']
['22']
['0']
['61']
['10']
['13']
['7']
['39']
['28']
['0']
['23']
['17']
['10']
['

['0']
['17']
['17']
['0']
['20']
['15']
['0']
['45']
['0']
['0']
['0']
['22']
['32']
['27']
['25']
['0']
['19']
['33']
['0']
['18']
['0']
['20']
['30']
['24']
['0']
['0']
['22']
['16']
['20']
['20']
['20']
['18']
['25']
['30']
['17']
['34']
['51']
['26']
['8']
['0']
['21']
['32']
['16']
['50']
['19']
['14']
['66']
['0']
['16']
['0']
['15']
['0']
['0']
['18']
['20.0']
['0']
['41']
['0']
['0']
['0']
['43']
['20']
['18']
['20']
['0']
['0']
['0']
['0']
['13']
['25']
['16']
['18']
['0']
['14']
['0']
['0']
['0']
['19']
['8']
['14']
['16']
['12']
['10']
['0']
['22']
['0']
['15']
['0']
['0']
['0']
['0']
['28']
['25']
['35']
['35']
['15']
['0']
['27']
['0']
['53']
['0']
['28']
['15']
['13']
['25']
['0']
['13']
['0']
['22']
['27']
['8']
['0']
['23']
['18']
['0']
['0']
[]
['65']
['16']
['49']
['52']
['21']
['0']
['0']
['0']
['27']
['17']
['0']
['28']
['30']
['22']
['16']
['0']
['0']
['0']
['0']
['24']
['9,12']
['15']
['12']
['41']
['18']
['25']
['33']
['33']
['0']
['43']
['17']
['17']
['0']
['0']

['23']
['53']
['14']
['22']
['31']
['0']
['0']
['0']
['0']
['0']
['40']
['0']
['0']
['0']
['0']
['0']
['38']
['0']
['27']
['22']
['20']
['0']
['0']
['0']
['5']
['13']
['19']
['14']
['12']
['38']
['0']
['0']
['0']
['0']
['0']
['0']
['18']
['11']
['0']
['0']
['8']
['34']
['0']
['12']
['15']
['17']
['0']
['0']
['0']
['0']
['15']
['22']
['0']
['17']
['0']
['0']
['0']
['0']
['0']
['50']
['0']
['15']
['0']
['0']
['0']
['0']
['0']
['0']
['24']
['0']
['0']
['0']
['0']
['38']
['0']
['0']
['0']
['24']
['27']
['0']
['0']
['0']
['0']
['0']
['18']
['17']
['0']
['32']
['0']
['0']
['0']
['0']
['0']
['0']
['0']
['0']
['0']
['0']
['0']
['36,23']
['23']
['45']
['13']
['0']
['20']
['0']
['18']
['0']
['0']
['23']
['0']
['0']
['0']
['0']
['0']
['0']
['0']
['0']
['0']
['0']
['0']
['19']
['0']
['0']
['39']
['0']
['0']
['18']
['0']
['0']
['49']
['0']
['0']
['0']
['0']
['0']
['0']
['17']
['38']
['0']
['29']
['21']
['0']
['0']
['0']
['0']
['17']
['0']
['19']
['18']
['0']
['0']
['23']
['0']
['17']
['0']
['16']
[

In [28]:
## CHAMP INJURY
#Etat des lieux
Counter(data['Injury']).most_common()

[('FATAL', 729),
 ('Survived', 97),
 ('Foot bitten', 82),
 ('No injury', 77),
 ('Leg bitten', 71),
 ('Left foot bitten', 48),
 ('No details', 43),
 ('Right foot bitten', 39),
 ('Hand bitten', 28),
 ('Thigh bitten', 27),
 ('No injury, board bitten', 27),
 (nan, 27),
 ('FATAL, body not recovered', 24),
 ('Minor injury', 22),
 ('Calf bitten', 21),
 ('Foot lacerated', 21),
 ('Right leg bitten', 20),
 ('Arm bitten', 19),
 ('Lacerations to foot', 19),
 ('Ankle bitten', 17),
 ('Right calf bitten', 16),
 ('Lacerations to right foot', 15),
 ('Lacerations to left foot', 14),
 ('No injury to occupants', 14),
 ('No injury, surfboard bitten', 13),
 ('Heel bitten', 13),
 ('Left arm bitten', 13),
 ('Left leg bitten', 13),
 ('Foot severed', 13),
 ('Right thigh bitten', 12),
 ('Leg lacerated', 12),
 ('Leg injured', 11),
 ('Left calf bitten', 11),
 ('Minor injuries', 11),
 ('Leg severed', 11),
 ('FATAL, leg severed ', 11),
 ('Lacerations to leg', 10),
 ('Lacerations to right leg', 10),
 ('Legs bitten', 

In [35]:
def remove_punctuation (text):
    cleanr = re.compile('\W',)
    cleantext_punctuation = re.sub(cleanr, ' ', text)
    return cleantext_punctuation

def minuscules (text):
    return text.lower()
    

# nettoie la liste, et agrège quelques expressions
data['Injury'] = data['Injury'].fillna('No details')
data['Injury'] = data['Injury'].apply(remove_punctuation).apply(minuscules)
liste_injury = [i.replace('\n',' ').replace('provoked incident','provoked-incident').replace('.',' ').replace(':',' ').replace(",",' ').replace("]",' ').replace('[',' ').replace("'",' ').replace('"',' ').replace('?',' ').replace(')',' ').replace('(',' ').replace('&','and').replace('/',' ').replace('no ','no-').split(' ') for i in data['Injury'].values]
liste_injury = list(filter(None, liste_injury))

#crée un bag of words
list_of_words = [y.lower() for x in liste_injury for y in x if y.lower() not in stop_words.ENGLISH_STOP_WORDS]
bag_of_words = list(set(list_of_words))

term_freq = []
for word in bag_of_words:
    term_freq.append (list_of_words.count (word))

#édition d'une liste des termes récurrents
mapped = zip(bag_of_words, term_freq)
getcount = operator.itemgetter(1)
map(getcount, mapped)
sortedlist = sorted(mapped, key=getcount, reverse=True)
print(sortedlist[1:])

[('bitten', 1535), ('shark', 1287), ('fatal', 1284), ('leg', 854), ('right', 810), ('left', 807), ('no-injury', 754), ('foot', 740), ('lacerations', 561), ('provoked-incident', 542), ('lacerated', 487), ('hand', 417), ('thigh', 402), ('arm', 388), ('severed', 315), ('calf', 263), ('laceration', 232), ('ankle', 227), ('boat', 220), ('minor', 212), ('bit', 203), ('lower', 189), ('occupants', 182), ('survived', 171), ('wounds', 163), ('puncture', 163), ('sharks', 161), ('recovered', 160), ('forearm', 158), ('knee', 147), ('hooked', 138), ('board', 138), ('injury', 134), ('injured', 129), ('s', 123), ('severely', 121), ('2', 119), ('body', 116), ('injuries', 109), ('legs', 103), ('3', 102), ('amputated', 92), ('shoulder', 88), ('surgically', 88), ('drowning', 87), ('surfboard', 81), ('remains', 77), ('death', 76), ('no-details', 76), ('torso', 73), ('heel', 72), ('severe', 71), ('involvement', 69), ('taken', 67), ('wrist', 65), ('occupant', 63), ('bumped', 62), ('abdomen', 59), ('rammed', 